In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

USER = 'aacuser'
PASS = 'SNHU2024'
HOST = 'nv-desktop-services.apporto.com'
PORT =  30109
DB =   'AAC'
COL =  'animals'
db = AnimalShelter(USER,PASS,HOST,PORT,DB,COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),height=256,width=256)),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.H3("Trent Hesler")),
    html.Hr(),
    html.Div(
        # Rescue type filter
        dcc.RadioItems(
            id='rescue-type',
            options=[
            {'label': 'Water Rescue', 'value': 'WaterRescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MountainRescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DisasterRescue'},
            {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    html.Div(
        # Breed matching precision selector
        dcc.RadioItems(
            id='include-mixes',
            options=[
            {'label': 'Include mixes', 'value': 'True'},
            {'label': 'Exclude mixes', 'value': 'False'}
            ],
            value='False',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    html.Hr(),
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        column_selectable=False,
        page_current=0,
        page_size=20,
        sort_action="native",
        filter_action="native"
    ),
    # Results counter
    html.Div(id='results-count'),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        # Pie chart
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        
        # Map
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Data table callback    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('rescue-type', 'value'),
               Input('include-mixes', 'value')])
def update_dashboard(rescue_type,include_mixes):
    if rescue_type == "Reset":
        df = pd.DataFrame.from_records(db.read({}))
    elif rescue_type == 'WaterRescue' :
        
        # Show inexact breed matches that include different breed
        if include_mixes == 'True':
            df = pd.DataFrame(db.read({'$and':[
                {'$or':[
                    {'breed':{'$regex':'Labrador Retriever Mix'}},
                    {'breed':{'$regex':'Chesa Bay Retr'}},
                    {'breed':{'$regex':'Newfoundland'}}
                ]},
                {'sex_upon_outcome':'Intact Female'},
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$gte':26}},
                    {'age_upon_outcome_in_weeks':{'$lte':156}}
                ]}
            ]}))

        # Only show exact breed matches
        else:
            df = pd.DataFrame(db.read({'$and':[
                {'$or':[
                    {'breed':'Labrador Retriever Mix'},
                    {'breed':'Chesa Bay Retr'},
                    {'breed':'Newfoundland'}
                ]},
                {'sex_upon_outcome':'Intact Female'},
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$gte':26}},
                    {'age_upon_outcome_in_weeks':{'$lte':156}}
                ]}
            ]}))

    elif rescue_type == 'MountainRescue' :
        
        # Show inexact breed matches that include different breed
        if include_mixes == 'True':
            df = pd.DataFrame.from_records(db.read({'$and':[
                {'$or':[
                    {'breed':{'$regex':'German Shepherd'}},
                    {'breed':{'$regex':'Alaskan Malamute'}},
                    {'breed':{'$regex':'Old English Sheepdog'}},
                    {'breed':{'$regex':'Siberian Husky'}},
                    {'breed':{'$regex':'Rottweiler'}}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$gte':26}},
                    {'age_upon_outcome_in_weeks':{'$lte':156}}
                ]}
            ]}))

        # Only show exact breed matches
        else:
            df = pd.DataFrame.from_records(db.read({'$and':[
                {'$or':[
                    {'breed':'German Shepherd'},
                    {'breed':'Alaskan Malamute'},
                    {'breed':'Old English Sheepdog'},
                    {'breed':'Siberian Husky'},
                    {'breed':'Rottweiler'}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$gte':26}},
                    {'age_upon_outcome_in_weeks':{'$lte':156}}
                ]}
            ]}))
        
    elif rescue_type == 'DisasterRescue' :
        # Show inexact breed matches that include different breed
        if include_mixes == 'True':
            df = pd.DataFrame.from_records(db.read({'$and':[
                {'$or':[
                    {'breed':{'$regex':'Doberman Pinscher'}},
                    {'breed':{'$regex':'German Shepherd'}},
                    {'breed':{'$regex':'Golden Retriever'}},
                    {'breed':{'$regex':'Bloodhound'}},
                    {'breed':{'$regex':'Rottweiler'}}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$gte':20}},
                    {'age_upon_outcome_in_weeks':{'$lte':300}}
                ]}
            ]}))

        # Only show exact breed matches
        else:
            df = pd.DataFrame.from_records(db.read({'$and':[
                {'$or':[
                    {'breed':'Doberman Pinscher'},
                    {'breed':'German Shepherd'},
                    {'breed':'Golden Retriever'},
                    {'breed':'Bloodhound'},
                    {'breed':'Rottweiler'}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                {'$and':[
                    {'age_upon_outcome_in_weeks':{'$gte':20}},
                    {'age_upon_outcome_in_weeks':{'$lte':300}}
                ]}
            ]}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)

# Graph callback
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)

    # Added if-else to stop callback "complaining" when first loaded
    if df.size < 1 :
        dcc.Graph(            
            figure = px.pie()
        )
    else:
        return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
        ]
    
# Row highlight callback
# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]


# Map callback
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        # Added try-except to stop callback from "complaining" when first loaded
        try:
            row = index[0]
        except:
            row = 0
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip((dff.iloc[row,9] if dff.iloc[row,9] != "" else "[No Name]")),
                dl.Popup([
                    html.H1(dff.iloc[row,9]),
                    html.H3("Breed: " + dff.iloc[row,4]),
                    html.H3("Sex: " + dff.iloc[row,12]),
                    html.H3("Age: {:.2f} weeks".format(dff.iloc[row,15]))
                ])
            ])
        ])
    ]

# Results count callback
# This callback counts how many results are returned based on the current filters
@app.callback(
    Output('results-count','children'),
    Input('datatable-id', "derived_virtual_data")
)
def update_results_count(viewData):
    dff = pd.DataFrame.from_dict(viewData)

    return[
        html.P("{} results".format(dff.size//16))
    ]


app.run_server(debug=True)

Dash app running on http://127.0.0.1:14091/
